<a href="https://colab.research.google.com/github/Prithwi13/6302_stock/blob/main/6302_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

first cell imports all the necessary libraries for your project. Most importantly, it securely loads your API keys from the Colab "Secrets" (🔑) tab.

In [25]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Optional
import time
import sys
from google.colab import userdata

# === SECURELY Load API keys from Colab Secrets ===
try:
    ALPHAVANTAGE_KEY = userdata.get('ALPHAVANTAGE_KEY')
    NEWSAPI_KEY = userdata.get('NEWSAPI_KEY')
    print("✓ API keys loaded successfully.")
except userdata.SecretNotFoundError:
    print("✗ FATAL ERROR: API Key not found in Colab Secrets.")
    print("  Please click the 'Key' icon (🔑) on the left sidebar,")
    print("  add 'ALPHAVANTAGE_KEY' and 'NEWSAPI_KEY',")
    print("  and make sure 'Notebook access' is toggled ON for both.")
    sys.exit(1) # Stop execution
except Exception as e:
    print(f"✗ FATAL ERROR: Could not load secrets. {e}")
    sys.exit(1)

✓ API keys loaded successfully.


This code runs the import statements and then immediately tries to fetch your keys from the Colab Secrets manager. If it fails, it prints a helpful error message and stops the script.

we define the classes responsible for fetching data from the two external APIs. These are your "worker" classes.

In [26]:
class StockPriceAPI:
    """Handles stock price data using Alpha Vantage"""
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://www.alphavantage.co/query"
        print(f"✓ Alpha Vantage initialized (API key: ...{api_key[-4:]})")

    def get_intraday_quotes(self, ticker: str, interval='5m', period='7d') -> pd.DataFrame:
        print(f"  ... [Alpha Vantage] Waiting 13s to respect 5 calls/min rate limit...")
        time.sleep(13)
        av_interval = interval.replace('m', 'min')
        av_outputsize = 'compact' if period == '1d' else 'full'
        params = {
            'function': 'TIME_SERIES_INTRADAY', 'symbol': ticker, 'interval': av_interval,
            'outputsize': av_outputsize, 'apikey': self.api_key, 'datatype': 'json'
        }
        try:
            response = requests.get(self.base_url, params=params, timeout=20)
            response.raise_for_status()
            data = response.json()
            if 'Error Message' in data:
                print(f"✗ [Alpha Vantage] API Error for {ticker}: {data['Error Message']}")
                return pd.DataFrame()
            if 'Note' in data:
                print(f"✗ [Alpha Vantage] API Note for {ticker}: {data['Note']}")
                return pd.DataFrame()
            data_key = next((key for key in data.keys() if 'Time Series' in key), None)
            if data_key is None:
                print(f"✗ [Alpha Vantage] Could not find 'Time Series' data key for {ticker}.")
                return pd.DataFrame()

            df = pd.DataFrame.from_dict(data[data_key], orient='index')
            if df.empty: return pd.DataFrame()

            df = df.reset_index().rename(columns={
                'index': 'timestamp', '1. open': 'open', '2. high': 'high',
                '3. low': 'low', '4. close': 'close', '5. volume': 'volume'
            })
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = pd.to_numeric(df[col])
            df['ticker'] = ticker
            try:
                df['timestamp'] = df['timestamp'].dt.tz_localize('America/New_York').dt.tz_convert('UTC')
            except Exception:
                df['timestamp'] = df['timestamp'].dt.tz_localize('UTC') # Fallback
            df = df[['timestamp', 'ticker', 'open', 'high', 'low', 'close', 'volume']]
            return df.sort_values('timestamp').reset_index(drop=True)
        except Exception as e:
            print(f"✗ Error fetching Alpha Vantage data for {ticker}: {e}")
            return pd.DataFrame()

class NewsAPI:
    """Handles financial news from NewsAPI"""
    def __init__(self, api_key: str):
        self.api_key = api_key
        print(f"✓ NewsAPI initialized (API key: ...{api_key[-4:]})")

    def get_news(self, ticker: str, company_name: str = None, days_back: int = 7) -> pd.DataFrame:
        url = 'https://newsapi.org/v2/everything'
        query = company_name if company_name else ticker
        from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
        params = {
            'q': query, 'from': from_date, 'sortBy': 'publishedAt',
            'language': 'en', 'apiKey': self.api_key, 'pageSize': 100
        }
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            if data['status'] != 'ok':
                raise ValueError(f"API Error: {data.get('message', 'Unknown error')}")

            records = [{
                'timestamp': pd.to_datetime(article['publishedAt']).tz_convert('UTC'),
                'headline': article['title'],
                'description': article.get('description', ''),
                'source': article['source']['name'],
                'url': article['url'],
                'ticker': ticker
            } for article in data.get('articles', []) if article.get('title')]

            return pd.DataFrame(records).sort_values('timestamp').reset_index(drop=True) if records else pd.DataFrame()
        except Exception as e:
            print(f"✗ Error fetching news for {ticker}: {e}")
            return pd.DataFrame()

print("✓ Data fetching classes defined.")

✓ Data fetching classes defined.


This cell defines the classes. It doesn't run them yet, it just makes them available for later use. Note the critical time.sleep(13) in StockPriceAPI—this is essential for not getting blocked by the free API.

These classes are the "brains" of the operation. IncrementalDataStorage is the most important part—it handles saving data and preventing duplicates. DataCollector manages the entire process.

In [27]:
class IncrementalDataStorage:
    """Handles incremental data storage and deduplication."""
    def __init__(self, data_dir='data/raw'):
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        self.prices_master_file = os.path.join(data_dir, 'prices_master.csv')
        self.news_master_file = os.path.join(data_dir, 'news_master.csv')
        self.backup_dir = os.path.join(data_dir, 'daily_backups')
        os.makedirs(self.backup_dir, exist_ok=True)

    def load_master_data(self) -> Dict[str, pd.DataFrame]:
        prices = pd.DataFrame()
        news = pd.DataFrame()
        if os.path.exists(self.prices_master_file):
            prices = pd.read_csv(self.prices_master_file)
            prices['timestamp'] = pd.to_datetime(prices['timestamp'])
            print(f"✓ Loaded {len(prices):,} existing price records")
        else:
            print("ℹ No existing price data found (starting fresh)")
        if os.path.exists(self.news_master_file):
            news = pd.read_csv(self.news_master_file)
            news['timestamp'] = pd.to_datetime(news['timestamp'])
            print(f"✓ Loaded {len(news):,} existing news records")
        else:
            print("ℹ No existing news data found (starting fresh)")
        return {'prices': prices, 'news': news}

    def append_new_data(self, new_data: Dict[str, pd.DataFrame]):
        existing = self.load_master_data()
        if not new_data['prices'].empty:
            combined_prices = pd.concat([existing['prices'], new_data['prices']], ignore_index=True)
            combined_prices = combined_prices.drop_duplicates(subset=['timestamp', 'ticker'], keep='last')
            combined_prices = combined_prices.sort_values(['ticker', 'timestamp']).reset_index(drop=True)
            combined_prices.to_csv(self.prices_master_file, index=False)
            print(f"✓ Saved {len(combined_prices):,} total price records (added {len(new_data['prices'])} new)")
        if not new_data['news'].empty:
            combined_news = pd.concat([existing['news'], new_data['news']], ignore_index=True)
            combined_news = combined_news.drop_duplicates(subset=['headline', 'ticker'], keep='first')
            combined_news = combined_news.sort_values(['ticker', 'timestamp']).reset_index(drop=True)
            combined_news.to_csv(self.news_master_file, index=False)
            print(f"✓ Saved {len(combined_news):,} total news records (added {len(new_data['news'])} new)")

    def get_statistics(self) -> dict:
        data = self.load_master_data()
        stats = {'total_price_records': len(data['prices']), 'total_news_articles': len(data['news']), 'tickers': [], 'date_range': None}
        if not data['prices'].empty:
            stats['tickers'] = data['prices']['ticker'].unique().tolist()
            stats['date_range'] = {'start': data['prices']['timestamp'].min(), 'end': data['prices']['timestamp'].max()}
            stats['records_per_ticker'] = data['prices'].groupby('ticker').size().to_dict()
        return stats

class DataCollector:
    """Main data collection orchestrator"""
    def __init__(self, price_api: StockPriceAPI, news_api: NewsAPI, storage: IncrementalDataStorage):
        self.price_api = price_api
        self.news_api = news_api
        self.storage = storage

    def collect_and_store(self, tickers: List[str], interval='5m', period='1d', news_days_back=1):
        all_prices, all_news = [], []
        company_map = {'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Google', 'AMZN': 'Amazon', 'TSLA': 'Tesla'}
        for ticker in tickers:
            print(f"\n📊 Collecting data for {ticker}...")
            prices = self.price_api.get_intraday_quotes(ticker, interval, period)
            if not prices.empty:
                all_prices.append(prices)
                print(f"  ✓ {len(prices)} price records")
            company_name = company_map.get(ticker, ticker)
            news = self.news_api.get_news(ticker, company_name, news_days_back)
            if not news.empty:
                all_news.append(news)
                print(f"  ✓ {len(news)} news articles")

        new_data = {
            'prices': pd.concat(all_prices, ignore_index=True) if all_prices else pd.DataFrame(),
            'news': pd.concat(all_news, ignore_index=True) if all_news else pd.DataFrame()
        }
        self.storage.append_new_data(new_data)
        return new_data

print("✓ Data storage and collector classes defined.")

✓ Data storage and collector classes defined.


This cell defines the classes that handle saving, loading, deduplicating, and orchestrating the entire collection process. Again, no data is fetched or saved yet.

Now we create "instances" of our classes. We pass the API keys to the fetching classes and create the storage and collector objects.

In [28]:
# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN']

# Initialize components
price_api = StockPriceAPI(ALPHAVANTAGE_KEY)
news_api = NewsAPI(NEWSAPI_KEY)
storage = IncrementalDataStorage('data/raw')
collector = DataCollector(price_api, news_api, storage)

print("\n✓ All components initialized and ready.")

✓ Alpha Vantage initialized (API key: ...Z82S)
✓ NewsAPI initialized (API key: ...d0ea)

✓ All components initialized and ready.


This is the final setup step. We've defined our list of TICKERS and created the objects we'll use in the next steps.

Let's check the status of our data files before we run the collection.

In [29]:
print("\n" + "="*70)
print("  STATUS BEFORE COLLECTION")
print("="*70)

stats_before = storage.get_statistics()
print(f"  Existing price records: {stats_before['total_price_records']:,}")
print(f"  Existing news articles: {stats_before['total_news_articles']:,}")


  STATUS BEFORE COLLECTION
ℹ No existing price data found (starting fresh)
✓ Loaded 460 existing news records
  Existing price records: 0
  Existing news articles: 460


We call the get_statistics() method on our storage object. On the very first run, this will create the data/raw directory and report that no data exists.

This is the main event. We call collector.collect_and_store(). This cell will take over a minute to run because of the 13-second pause for each of the 5 tickers (5 * 13 = 65 seconds).

In [30]:
print("\n" + "="*70)
print("  COLLECTING TODAY'S DATA... (This will take > 1 minute)")
print("="*70)

new_data = collector.collect_and_store(
    tickers=TICKERS,
    interval='5m',
    period='1d',      # IMPORTANT: Only get today's data!
    news_days_back=1  # Only today's news
)

print("\n" + "="*70)
print("  ✅ COLLECTION COMPLETE!")
print("="*70)


  COLLECTING TODAY'S DATA... (This will take > 1 minute)

📊 Collecting data for AAPL...
  ... [Alpha Vantage] Waiting 13s to respect 5 calls/min rate limit...
  ✓ 100 price records
  ✓ 93 news articles

📊 Collecting data for MSFT...
  ... [Alpha Vantage] Waiting 13s to respect 5 calls/min rate limit...
  ✓ 100 price records
  ✓ 96 news articles

📊 Collecting data for GOOGL...
  ... [Alpha Vantage] Waiting 13s to respect 5 calls/min rate limit...
  ✓ 100 price records
  ✓ 100 news articles

📊 Collecting data for TSLA...
  ... [Alpha Vantage] Waiting 13s to respect 5 calls/min rate limit...
  ✓ 100 price records
  ✓ 78 news articles

📊 Collecting data for AMZN...
  ... [Alpha Vantage] Waiting 13s to respect 5 calls/min rate limit...
  ✓ 100 price records
  ✓ 96 news articles
ℹ No existing price data found (starting fresh)
✓ Loaded 460 existing news records
✓ Saved 500 total price records (added 500 new)
✓ Saved 460 total news records (added 463 new)

  ✅ COLLECTION COMPLETE!


The collector will now loop through each ticker. For each one, it will:

Call price_api.get_intraday_quotes() (pausing for 13 seconds).

Call news_api.get_news().

After the loop, it will call storage.append_new_data() to save the results.

Now that Cell 6 has finished, let's check the stats again. The numbers should now reflect the data we just downloaded.

In [31]:
print("\n" + "="*70)
print("  STATUS AFTER COLLECTION")
print("="*70)

stats_after = storage.get_statistics()
print(f"  Total price records: {stats_after['total_price_records']:,}")
print(f"  Total news articles: {stats_after['total_news_articles']:,}")
if stats_after.get('date_range'):
    print(f"  Date range: {stats_after['date_range']['start']} to {stats_after['date_range']['end']}")
if stats_after.get('records_per_ticker'):
    print("\n  Records per ticker:")
    for ticker, count in stats_after['records_per_ticker'].items():
        print(f"    • {ticker}: {count:,} records")


  STATUS AFTER COLLECTION
✓ Loaded 500 existing price records
✓ Loaded 460 existing news records
  Total price records: 500
  Total news articles: 460
  Date range: 2025-11-07 16:40:00+00:00 to 2025-11-08 00:55:00+00:00

  Records per ticker:
    • AAPL: 100 records
    • AMZN: 100 records
    • GOOGL: 100 records
    • MSFT: 100 records
    • TSLA: 100 records


Running get_statistics() now will find the prices_master.csv and news_master.csv files, load them, and report the new counts.

The stats are great, but let's look at the actual data we saved to confirm it's correct.

In [32]:
def view_sample_data():
    """Helper function to view accumulated data"""
    print("\n" + "="*70)
    print("  ACCUMULATED DATA VIEWER")
    print("="*70)
    storage = IncrementalDataStorage('data/raw')
    data = storage.load_master_data()

    if not data['prices'].empty:
        print("\n  📊 Sample Price Data (latest 5 records):")
        print(data['prices'].tail(5)[['timestamp', 'ticker', 'close', 'volume']])
    else:
        print("\n  ℹ No price data to display.")

    if not data['news'].empty:
        print("\n  📰 Sample News Data (latest 5 articles):")
        for _, row in data['news'].tail(5).iterrows():
            print(f"    • [{row['ticker']}] {row['headline'][:70]}...")
    else:
        print("\n  ℹ No news data to display.")
    print("\n" + "="*70 + "\n")

# Run the view function
view_sample_data()


  ACCUMULATED DATA VIEWER
✓ Loaded 500 existing price records
✓ Loaded 460 existing news records

  📊 Sample Price Data (latest 5 records):
                    timestamp ticker   close  volume
495 2025-11-08 00:35:00+00:00   TSLA  432.23    6208
496 2025-11-08 00:40:00+00:00   TSLA  431.99   12163
497 2025-11-08 00:45:00+00:00   TSLA  432.00   21640
498 2025-11-08 00:50:00+00:00   TSLA  432.05   30719
499 2025-11-08 00:55:00+00:00   TSLA  432.03   28893

  📰 Sample News Data (latest 5 articles):
    • [TSLA] Elon Musk uses Grok to imagine the possibility of love | TechCrunch...
    • [TSLA] Vibe-coding is now an official word in the dictionary...
    • [TSLA] Elon Musk uses Grok to imagine the possibility of love...
    • [TSLA] Got a spare $50,000? Cooling a single Nvidia Blackwell Ultra NVL72 rac...
    • [TSLA] Mark Cuban Said 'I Didn't Care About The Business Side Of The Maverick...




This cell defines and runs a simple function to load the master files and print the last 5 rows of each DataFrame. This is the best way to verify that the data looks correct.